In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from IPython.display import clear_output
from pydrake.all import (
    AbstractValue,
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    JointSliders,
    LeafSystem,
    MeshcatPoseSliders,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    PointCloud,
    RandomGenerator,
    Rgba,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
)
from manipulation.scenarios import AddMultibodyTriad

from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingRpyJoint, AddRgbdSensors, ycb
from manipulation.utils import ConfigureParser
import sponana.utils

In [3]:
from sponana.grasping.grasp_generator import (
    get_unified_point_cloud,
    BananaSystem,
    ScoreSystem,
    GenerateAntipodalGraspCandidate,
)

In [4]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [7]:
import sponana.grasping.grasping_models as g


def make_internal_model():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)
    parser.AddModelsFromString(
        g.spot_and_target_str(
            target_obj_path=g.SUGAR_BOX_TARGET_PATH,
            target_obj_link=g.SUGAR_BOX_TARGET_LINK,
        ),
        "dmd.yaml",
    )
    # AddModelsFromUrl("package://sponana/grasping/banana_and_spot_gripper.dmd.yaml")
    plant.Finalize()
    return builder.Build()

In [16]:
def grasp_score_inspector(gripper_name="arm_link_wr1"):
    meshcat.Delete()

    # Finally, we'll build a diagram for running our visualization
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)

    parser.AddModelsFromString(
        g.gripper_and_target_str(
            target_obj_path=g.SUGAR_BOX_TARGET_PATH,
            target_obj_link=g.SUGAR_BOX_TARGET_LINK,
        ),
        "dmd.yaml",
    )

    AddFloatingRpyJoint(
        plant,
        plant.GetFrameByName(gripper_name),
        plant.GetModelInstanceByName("_gripper"),
    )
    AddMultibodyTriad(plant.GetFrameByName(gripper_name), scene_graph)
    plant.Finalize()

    meshcat.DeleteAddedControls()
    params = MeshcatVisualizerParams()
    params.prefix = "planning"
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, params)

    environment = BananaSystem(
        g.target_and_cameras_str(
            target_obj_path=g.SUGAR_BOX_TARGET_PATH,
            target_obj_link=g.SUGAR_BOX_TARGET_LINK,
        )
    )
    environment_context = environment.CreateDefaultContext()

    cloud = get_unified_point_cloud(environment, environment_context, meshcat=meshcat)
    meshcat.SetObject("planning/cloud", cloud, point_size=0.003)

    internal_model = make_internal_model()
    score = builder.AddSystem(
        ScoreSystem(
            internal_model,
            cloud,
            gripper_name,
            plant.GetBodyByName(gripper_name).index(),
            meshcat=meshcat,
        )
    )
    builder.Connect(plant.get_body_poses_output_port(), score.get_input_port())

    lower_limit = [-1, -1, 0, -np.pi, -np.pi / 4.0, -np.pi / 4.0]
    upper_limit = [1, 1, 1, 0, np.pi / 4.0, np.pi / 4.0]
    q0 = [-0.05, -0.5, 0.25, -np.pi / 2.0, 0, 0]
    default_interactive_timeout = None if running_as_notebook else 1.0
    sliders = builder.AddSystem(
        JointSliders(
            meshcat,
            plant,
            initial_value=q0,
            lower_limit=lower_limit,
            upper_limit=upper_limit,
            decrement_keycodes=[
                "KeyQ",
                "KeyS",
                "KeyA",
                "KeyJ",
                "KeyK",
                "KeyU",
            ],
            increment_keycodes=[
                "KeyE",
                "KeyW",
                "KeyD",
                "KeyL",
                "KeyI",
                "KeyO",
            ],
        )
    )
    diagram = builder.Build()
    sliders.Run(diagram, default_interactive_timeout)
    meshcat.DeleteAddedControls()


grasp_score_inspector(gripper_name="gripper")  # gripper_name="body")

RuntimeError: GetBodyByName(): There is no Body named 'gripper' anywhere in the model (valid names are: arm_link_el0, arm_link_el1, arm_link_fngr, arm_link_hr0, arm_link_sh0, arm_link_sh1, arm_link_wr0, arm_link_wr1, base_link, base_link_sugar, base_x, base_y, base_z, body, front_left_hip, front_left_lower_leg, front_left_upper_leg, front_rail, front_right_hip, front_right_lower_leg, front_right_upper_leg, rear_left_hip, rear_left_lower_leg, rear_left_upper_leg, rear_rail, rear_right_hip, rear_right_lower_leg, rear_right_upper_leg, table_top_link, world)

# Generate grasps

In [11]:
# For visualization
def draw_grasp_candidate(
    X_G, meshcat, gripper_name, prefix="gripper", draw_frames=True
):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)
    parser.AddModelsFromUrl("package://sponana/grasping/spot_gripper.urdf")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName(gripper_name), X_G)
    plant.Finalize()

    # frames_to_draw = {"gripper": {"body"}} if draw_frames else {}
    params = MeshcatVisualizerParams()
    params.prefix = prefix
    params.delete_prefix_on_initialization_event = False
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, params)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.ForcedPublish(context)

In [13]:
def sample_grasps_example(
    gripper_name="arm_link_wr1", pointcloud_transform=RigidTransform()
):
    meshcat.Delete()
    rng = np.random.default_rng()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    sponana.utils.configure_parser(parser)

    parser.AddModelsFromString(
        g.gripper_and_target_str(
            target_obj_path=g.SUGAR_BOX_TARGET_PATH,
            target_obj_link=g.SUGAR_BOX_TARGET_LINK,
        ),
        "dmd.yaml",
    )

    AddFloatingRpyJoint(
        plant,
        plant.GetFrameByName(gripper_name),
        plant.GetModelInstanceByName("_gripper"),
    )
    plant.Finalize()

    params = MeshcatVisualizerParams()
    params.prefix = "planning"
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat, params)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.ForcedPublish(context)

    # Hide the planning gripper
    meshcat.SetProperty("planning/gripper", "visible", False)

    environment = BananaSystem(
        g.target_and_cameras_str(
            target_obj_path=g.SUGAR_BOX_TARGET_PATH,
            target_obj_link=g.SUGAR_BOX_TARGET_LINK,
        )
    )
    environment_context = environment.CreateDefaultContext()
    cloud = get_unified_point_cloud(environment, environment_context, meshcat=meshcat)
    cloud = pointcloud_transform.multiply(cloud)
    meshcat.SetObject("planning/cloud", cloud, point_size=0.003)

    plant.GetMyContextFromRoot(context)
    scene_graph.GetMyContextFromRoot(context)

    internal_model = make_internal_model()
    internal_model_context = internal_model.CreateDefaultContext()
    costs = []
    X_Gs = []
    for i in range(1000 if running_as_notebook else 2):
        cost, X_G = GenerateAntipodalGraspCandidate(
            internal_model,
            internal_model_context,
            cloud,
            rng,
            wsg_body_index=plant.GetBodyByName(gripper_name).index(),
        )
        if np.isfinite(cost):
            costs.append(cost)
            X_Gs.append(X_G)

    indices = np.asarray(costs).argsort()[:5]
    for rank, index in enumerate(indices):
        draw_grasp_candidate(
            X_Gs[index],
            meshcat,
            gripper_name,
            prefix=f"{rank}th best",
            draw_frames=False,
        )

    print(costs)


sample_grasps_example(gripper_name="gripper")

TypeError: multiply(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.math.RigidTransform, other: pydrake.math.RigidTransform) -> pydrake.math.RigidTransform
    2. (self: pydrake.math.RigidTransform, p_BoQ_B: numpy.ndarray[numpy.float64[3, 1]]) -> numpy.ndarray[numpy.float64[3, 1]]
    3. (self: pydrake.math.RigidTransform, vec_B: numpy.ndarray[numpy.float64[4, 1]]) -> numpy.ndarray[numpy.float64[4, 1]]
    4. (self: pydrake.math.RigidTransform, p_BoQ_B: numpy.ndarray[numpy.float64[3, n]]) -> numpy.ndarray[numpy.float64[3, n]]

Invoked with: RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
), <pydrake.perception.PointCloud object at 0x2a065f430>